In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from scipy import stats
import statistics
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_error, make_scorer
import seaborn as sns
import os
from scipy.stats import skew, probplot, norm
import sklearn.model_selection as ms
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, Lasso, ElasticNet 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor

function to visualize the frequency of each unique value in each feature

In [2]:
#function to summarize feature
def summarize_feature(dataframe,feature):
    feature_summary = {'values':dataframe[feature].value_counts().index.tolist(),
                     'counts':dataframe[feature].value_counts().values.tolist(),
                     'frequency': dataframe[feature].value_counts(normalize = True).values.tolist()}
    feature_summary = pd.DataFrame(feature_summary)
    return feature_summary


# XGBOOST modeling

### importing data, whole train

In [39]:
train_df = pd.read_csv('./standardized splits/30/unique_train_cleaned_30.csv')
test_df = pd.read_csv('./standardized splits/30/unique_test_cleaned_30.csv')

print(f'unique_train_cleaned_30 dataset has {train_df.shape}')
print(f'unique_test_cleaned_30 dataset has {test_df.shape}')


unique_train_cleaned_30 dataset has (52825, 94)
unique_test_cleaned_30 dataset has (25902, 94)


In [40]:
#dropping the columns encounter_id and patient_nbr
train_df = train_df.drop(['encounter_id','patient_nbr'], axis = 1)
test_df = test_df.drop(['encounter_id','patient_nbr'], axis = 1)


In [41]:
train_df.sample(5)

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,...,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes
45399,1.207636,-0.127836,0.616617,-0.783594,-0.615110,1.288826,-0.2116,-0.500531,0.823365,0,...,0,1,0,0,0,0,0,0,1,1
247,1.207636,0.882750,0.565623,0.390898,-0.244522,-0.291878,-0.2116,-0.500531,0.308175,1,...,0,1,0,0,0,0,0,0,1,1
40257,-1.300937,-1.138422,-1.015207,-0.783594,-1.109228,-0.291878,-0.2116,-0.500531,-0.722205,0,...,0,1,0,0,0,0,0,0,1,0
19104,-1.300937,-0.801560,-2.137087,-0.196348,-1.109228,0.498474,-0.2116,-0.500531,-1.752584,0,...,0,1,0,0,0,0,0,0,1,1
30905,0.580493,0.882750,-0.250290,-0.783594,-0.615110,-0.291878,-0.2116,1.086189,-1.752584,0,...,0,1,0,0,0,0,0,0,1,1


In [42]:
summarize_feature(train_df, 'readmitted')

,values,counts,frequency
0,0,48516,0.918429
1,1,4309,0.081571


In [43]:
summarize_feature(test_df, 'readmitted')

,values,counts,frequency
0,0,23419,0.904139
1,1,2483,0.095861


In [8]:
#set up the train data set into X and y
X = train_df.loc[:, train_df.columns !='readmitted']
y = train_df[['readmitted']]


In [9]:
#set up the test data into X_test and y_test
X_test = test_df.loc[:, test_df.columns !='readmitted']
y_test = test_df[['readmitted']]


In [10]:
# Apply smote technique to balance the train data
from imblearn.over_sampling import SMOTE

from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
sm = SMOTE(random_state=20)
X_new, y_new = sm.fit_sample(X, y)
print('New dataset shape {}'.format(Counter(y_new)))

Using TensorFlow backend.


Original dataset shape Counter({'readmitted': 1})
New dataset shape Counter({0: 48516, 1: 48516})


In [11]:
#Make X_new back into a df instead of a numpy array
X_new = pd.DataFrame(X_new, columns = list(X.columns))

In [12]:
# split train data set in train and valid sets for model training
X_train, X_valid, y_train, y_valid = train_test_split(X_new, y_new, test_size=0.20, random_state=42)


In [13]:
import xgboost as xgb


In [14]:
xgbc = xgb.XGBClassifier()

In [15]:
#fit X_train, y_train
xgbc.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [16]:
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
# def calc_specificity(y_actual, y_pred, thresh):
#     # calculates specificity
#     return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

# def print_report(y_actual, y_pred, thresh):
    
#     auc = roc_auc_score(y_actual, y_pred)
#     accuracy = accuracy_score(y_actual, (y_pred > thresh))
#     recall = recall_score(y_actual, (y_pred > thresh))
#     precision = precision_score(y_actual, (y_pred > thresh))
#     specificity = calc_specificity(y_actual, y_pred, thresh)
#     print('AUC:%.3f'%auc)
#     print('accuracy:%.3f'%accuracy)
#     print('recall:%.3f'%recall)
#     print('precision:%.3f'%precision)
#     print('specificity:%.3f'%specificity)
#     print('prevalence:%.3f'%calc_prevalence(y_actual))
#     print(' ')
#     return auc, accuracy, recall, precision, specificity

# def calc_prevalence(y_actual):
#     return (sum(y_actual)/len(y_actual))

In [16]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix

def print_report(y_actual, y_pred, y_pred_proba):
    tp, fp, fn, tn = confusion_matrix(y_actual, y_pred).ravel()
    fpr = fp/(fp+tn)
    auc = roc_auc_score(y_actual, y_pred_proba)
    print('AUC:%.3f'%auc)
    print(f'False positive rate is {fpr}')
    print(' ')
    
#     return auc, fpr


In [17]:
thresh=0.5
y_train_pred_proba = xgbc.predict_proba(X_train)[:,1]
y_valid_pred_proba = xgbc.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc.predict_proba(X_test)[:,1]

y_train_pred = xgbc.predict(X_train)
y_valid_pred = xgbc.predict(X_valid)
y_test_pred = xgbc.predict(X_test)

print('XGBoost')
print('Training:')
print_report(y_train, y_train_pred, y_train_pred_proba)
print('Validation:')
print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)


XGBoost
Training:
AUC:0.968
False positive rate is 0.005890846087207624
 
Validation:
AUC:0.968
False positive rate is 0.008269275564873049
 
Test:
AUC:0.633
False positive rate is 0.815668202764977
 


In [18]:
sorted(zip(X_train.columns,xgbc.feature_importances_), key= lambda t:t[1], reverse=True)


[('gender_Male', 0.086914375),
 ('A1Cresult_None', 0.07863705),
 ('diag_1_neoplasms', 0.06834941),
 ('discharge_disposition_id_home', 0.06400882),
 ('diag_1_respiratory', 0.052130267),
 ('race_Caucasian', 0.05046313),
 ('diag_1_digestive', 0.04399486),
 ('age', 0.043494612),
 ('time_in_hospital', 0.042303458),
 ('admission_type_id_urgent', 0.041324027),
 ('number_inpatient', 0.034621585),
 ('num_procedures', 0.034435455),
 ('diag_1_other', 0.03384887),
 ('diag_1_genitourinary', 0.030622428),
 ('discharge_disposition_id_na', 0.023144718),
 ('number_outpatient', 0.023034666),
 ('number_diagnoses', 0.022493578),
 ('insulin_No', 0.019943252),
 ('A1Cresult_>8', 0.016597651),
 ('insulin_Steady', 0.01633881),
 ('pioglitazone_No', 0.016055627),
 ('admission_source_id_urgent', 0.0143523095),
 ('diag_1_injury', 0.0140235415),
 ('diag_1_diabetes', 0.0136692785),
 ('discharge_disposition_id_hospital', 0.013433454),
 ('metformin_No', 0.012909271),
 ('diag_1_musculoskeletal', 0.012370239),
 ('number

## parameter tuning

In [19]:
xgbc1 = xgb.XGBClassifier()
xgbc1.set_params(learning_rate=0.1,subsample=0.8,n_estimators=3000,max_depth=6,
                colsample_bytree=1,objective='binary:logistic')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [20]:
#fit X_train, y_train
xgbc1.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [27]:
xgbc1.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)>

In [28]:
thresh=0.5
y_train_pred_proba = xgbc1.predict_proba(X_train)[:,1]
y_valid_pred_proba = xgbc1.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc1.predict_proba(X_test)[:,1]

y_train_pred = xgbc1.predict(X_train)
y_valid_pred = xgbc1.predict(X_valid)
y_test_pred = xgbc1.predict(X_test)

print('Tuned XGBoost')
print('Training:')
print_report(y_train, y_train_pred, y_train_pred_proba)
print('Validation:')
print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)


Tuned XGBoost
Training:
AUC:1.000
False positive rate is 2.5746652935118435e-05
 
Validation:
AUC:0.967
False positive rate is 0.015572484875644185
 
Test:
AUC:0.595
False positive rate is 0.7535885167464115
 


In [29]:
sorted(zip(X_train.columns,xgbc1.feature_importances_), key= lambda t:t[1], reverse=True)


[('discharge_disposition_id_home', 0.06871399),
 ('number_inpatient', 0.06618908),
 ('race_Caucasian', 0.04988853),
 ('A1Cresult_None', 0.039659776),
 ('gender_Male', 0.036024526),
 ('diag_1_respiratory', 0.033392355),
 ('discharge_disposition_id_hospital', 0.032423627),
 ('age', 0.031520486),
 ('discharge_disposition_id_na', 0.029182794),
 ('diag_1_other', 0.024885815),
 ('diag_1_digestive', 0.02406854),
 ('diabetesMed_Yes', 0.023830026),
 ('diag_1_neoplasms', 0.023428027),
 ('time_in_hospital', 0.019936709),
 ('diag_1_genitourinary', 0.01919183),
 ('diag_1_injury', 0.018561646),
 ('number_diagnoses', 0.016321665),
 ('num_procedures', 0.016280621),
 ('glimepiride_Steady', 0.01597332),
 ('admission_type_id_urgent', 0.015757978),
 ('pioglitazone_Steady', 0.015702289),
 ('diag_1_diabetes', 0.014206566),
 ('diag_1_musculoskeletal', 0.01376666),
 ('admission_source_id_transfer', 0.012640475),
 ('admission_source_id_urgent', 0.011597336),
 ('rosiglitazone_Up', 0.011341949),
 ('metformin_No'

In [30]:
import eli5
eli5.show_weights(xgbc1)


Weight,Feature
0.0687,discharge_disposition_id_home
0.0662,number_inpatient
0.0499,race_Caucasian
0.0397,A1Cresult_None
0.0360,gender_Male
0.0334,diag_1_respiratory
0.0324,discharge_disposition_id_hospital
0.0315,age
0.0292,discharge_disposition_id_na
0.0249,diag_1_other


In [31]:
X_test['predicted_probaility'] = y_test_pred_proba

In [32]:
X_test['predicted_probaility'] = y_test_pred_proba
X_test['actual_readmitted'] = y_test
X_test['predicted_readmitted'] = y_test_pred

In [33]:
X_test.shape

(25902, 94)

In [34]:
X_test.sort_values(by='predicted_probaility')

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,med_dosage_change,...,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes,predicted_probaility,actual_readmitted,predicted_readmitted
16360,-0.046651,2.903921,-1.117197,-0.783594,-0.615110,-0.291878,-0.211600,-0.500531,-1.752584,1,...,0,0,0,0,0,0,1,1.267097e-07,0,0
16035,-1.928080,-0.801560,0.667612,0.390898,-0.738640,-0.291878,-0.211600,-0.500531,-2.267774,0,...,0,0,0,0,0,1,0,4.638023e-07,0,0
19116,-3.182366,-0.464698,-2.137087,0.390898,-0.491581,0.498474,-0.211600,-0.500531,-0.722205,0,...,0,0,0,0,0,1,0,5.191774e-07,0,0
18494,-0.673794,-0.127836,-2.137087,2.152636,-0.244522,1.288826,-0.211600,-0.500531,-0.207015,0,...,0,0,0,0,0,0,1,5.988628e-07,0,0
25158,-0.046651,-0.464698,-0.046311,-0.783594,-1.850405,-0.291878,-0.211600,-0.500531,-0.207015,0,...,0,0,0,0,0,1,0,6.065337e-07,0,0
11952,-0.046651,-1.138422,-1.831120,-0.783594,-1.356287,-0.291878,-0.211600,-0.500531,-1.237394,0,...,0,0,0,0,0,0,1,7.331438e-07,1,0
21035,-3.182366,-1.138422,-1.168191,-0.783594,-1.356287,-0.291878,-0.211600,-0.500531,-2.782964,0,...,0,0,0,0,0,1,1,7.566819e-07,0,0
21898,-0.046651,-1.138422,-2.137087,0.390898,0.496655,-0.291878,-0.211600,-0.500531,-0.207015,0,...,0,0,0,0,0,1,1,7.581901e-07,0,0
3692,-1.928080,-0.464698,0.463634,-0.783594,-1.356287,0.498474,-0.211600,-0.500531,-1.752584,1,...,0,0,0,0,0,0,1,8.511334e-07,0,0
15798,-1.300937,-0.464698,-1.729131,1.565390,0.002537,0.498474,-0.211600,-0.500531,-1.237394,0,...,0,0,0,0,0,1,0,8.720033e-07,0,0


In [35]:
#adding the quantiles as another columns
X_test['Quantile_rank'] = pd.qcut(X_test['predicted_probaility'],10,labels=False)


In [36]:
X_test

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,med_dosage_change,...,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes,predicted_probaility,actual_readmitted,predicted_readmitted,Quantile_rank
0,0.580493,-0.801560,-0.556257,-0.196348,1.855480,-0.291878,-0.211600,0.292829,-1.237394,0,...,0,0,0,0,0,1,0.266360,0,0,9
1,1.207636,2.567059,-1.321175,-0.783594,0.496655,0.498474,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,0,1,0.000253,0,0,0
2,0.580493,-1.138422,-0.658246,2.739882,0.496655,-0.291878,-0.211600,0.292829,0.823365,0,...,0,0,0,0,1,0,0.004222,0,0,3
3,-3.809509,-0.801560,-0.913218,-0.783594,-1.726876,-0.291878,-0.211600,-0.500531,-2.782964,0,...,0,0,0,0,1,0,0.000042,0,0,0
4,-0.046651,0.882750,-0.658246,-0.783594,-0.244522,-0.291878,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,0,1,0.002356,0,0,2
5,0.580493,-0.127836,0.514628,0.390898,-0.491581,-0.291878,-0.211600,0.292829,0.823365,0,...,0,0,0,0,1,1,0.029518,0,0,6
6,0.580493,-0.127836,-0.046311,0.978144,1.237832,-0.291878,-0.211600,-0.500531,-0.722205,0,...,0,0,0,0,1,1,0.026165,0,0,6
7,-0.046651,-0.464698,0.463634,2.152636,0.249596,-0.291878,-0.211600,-0.500531,0.308175,0,...,0,0,0,0,1,1,0.002489,0,0,2
8,0.580493,-0.801560,-1.576147,-0.783594,-0.738640,-0.291878,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,1,1,0.456796,0,0,9
9,0.580493,1.219612,1.789492,-0.196348,0.126067,-0.291878,-0.211600,0.292829,0.823365,1,...,0,0,0,0,0,1,0.007908,0,0,4


In [37]:
X_test.groupby('Quantile_rank')['predicted_probaility'].agg(['min','max','mean'])

,min,max,mean
Quantile_rank,,,
0,1.267097e-07,0.000426,0.000173
1,4.256839e-04,0.001314,0.000825
2,1.314024e-03,0.002914,0.002035
3,2.916162e-03,0.005497,0.004095
4,5.497600e-03,0.009606,0.007377
5,9.606421e-03,0.016674,0.012828
6,1.667414e-02,0.029545,0.022504
7,2.954506e-02,0.056719,0.041114
8,5.675879e-02,0.130073,0.086393


### importing data

In [44]:
train_df = pd.read_csv('./standardized splits/30/whole_train_cleaned_30.csv')
test_df = pd.read_csv('./standardized splits/30/unique_test_cleaned_30.csv')

print(f'whole_train_cleaned_30 dataset has {train_df.shape}')
print(f'unique_test_cleaned_30 dataset has {test_df.shape}')


whole_train_cleaned_30 dataset has (69524, 94)
unique_test_cleaned_30 dataset has (25902, 94)


In [45]:
#dropping the columns encounter_id and patient_nbr
train_df = train_df.drop(['encounter_id','patient_nbr'], axis = 1)
test_df = test_df.drop(['encounter_id','patient_nbr'], axis = 1)


In [46]:
train_df.sample(5)

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,...,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes
58490,-0.046651,-0.464698,0.361645,-0.783594,0.990773,-0.291878,-0.2116,3.466268,0.308175,0,...,0,1,0,0,0,0,0,0,0,1
7448,1.834779,0.882750,0.055678,-0.196348,0.126067,-0.291878,-0.2116,0.292829,-1.237394,0,...,0,1,0,0,0,0,0,0,1,0
68792,1.207636,0.882750,0.973579,-0.196348,-0.368051,0.498474,-0.2116,-0.500531,0.823365,1,...,0,1,0,0,0,0,0,0,1,1
23958,1.207636,-0.801560,-0.301284,0.390898,-0.244522,-0.291878,-0.2116,-0.500531,-2.267774,0,...,0,1,0,0,0,0,0,0,0,1
55919,-1.928080,-0.464698,-0.148301,0.390898,1.731950,-0.291878,-0.2116,-0.500531,-0.207015,0,...,0,1,0,0,0,0,0,0,1,0


In [47]:
summarize_feature(train_df, 'readmitted')

,values,counts,frequency
0,0,61608,0.88614
1,1,7916,0.11386


In [48]:
summarize_feature(test_df, 'readmitted')

,values,counts,frequency
0,0,23419,0.904139
1,1,2483,0.095861


In [49]:
#set up the train data set into X and y
X = train_df.loc[:, train_df.columns !='readmitted']
y = train_df[['readmitted']]


In [50]:
#set up the test data into X_test and y_test
X_test = test_df.loc[:, test_df.columns !='readmitted']
y_test = test_df[['readmitted']]


In [51]:
# Apply smote technique to balance the train data
from imblearn.over_sampling import SMOTE

from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
sm = SMOTE(random_state=20)
X_new, y_new = sm.fit_sample(X, y)
print('New dataset shape {}'.format(Counter(y_new)))

Original dataset shape Counter({'readmitted': 1})
New dataset shape Counter({0: 61608, 1: 61608})


In [52]:
#Make X_new back into a df instead of a numpy array
X_new = pd.DataFrame(X_new, columns = list(X.columns))

In [53]:
# split train data set in train and valid sets for model training
X_train, X_valid, y_train, y_valid = train_test_split(X_new, y_new, test_size=0.20, random_state=42)


In [54]:
import xgboost as xgb


In [55]:
xgbc = xgb.XGBClassifier()

In [56]:
#fit X_train, y_train
xgbc.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [57]:
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
# def calc_specificity(y_actual, y_pred, thresh):
#     # calculates specificity
#     return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

# def print_report(y_actual, y_pred, thresh):
    
#     auc = roc_auc_score(y_actual, y_pred)
#     accuracy = accuracy_score(y_actual, (y_pred > thresh))
#     recall = recall_score(y_actual, (y_pred > thresh))
#     precision = precision_score(y_actual, (y_pred > thresh))
#     specificity = calc_specificity(y_actual, y_pred, thresh)
#     print('AUC:%.3f'%auc)
#     print('accuracy:%.3f'%accuracy)
#     print('recall:%.3f'%recall)
#     print('precision:%.3f'%precision)
#     print('specificity:%.3f'%specificity)
#     print('prevalence:%.3f'%calc_prevalence(y_actual))
#     print(' ')
#     return auc, accuracy, recall, precision, specificity

# def calc_prevalence(y_actual):
#     return (sum(y_actual)/len(y_actual))

In [58]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix

def print_report(y_actual, y_pred, y_pred_proba):
    tp, fp, fn, tn = confusion_matrix(y_actual, y_pred).ravel()
    fpr = fp/(fp+tn)
    auc = roc_auc_score(y_actual, y_pred_proba)
    print('AUC:%.3f'%auc)
    print(f'False positive rate is {fpr}')
    print(' ')
    
#     return auc, fpr


In [59]:
thresh=0.5
y_train_pred_proba = xgbc.predict_proba(X_train)[:,1]
y_valid_pred_proba = xgbc.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc.predict_proba(X_test)[:,1]

y_train_pred = xgbc.predict(X_train)
y_valid_pred = xgbc.predict(X_valid)
y_test_pred = xgbc.predict(X_test)

print('XGBoost')
print('Training:')
print_report(y_train, y_train_pred, y_train_pred_proba)
print('Validation:')
print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)


XGBoost
Training:
AUC:0.953
False positive rate is 0.013022731012435063
 
Validation:
AUC:0.951
False positive rate is 0.013864983744501817
 
Test:
AUC:0.631
False positive rate is 0.7058823529411765
 


In [60]:
sorted(zip(X_train.columns,xgbc.feature_importances_), key= lambda t:t[1], reverse=True)


[('A1Cresult_None', 0.08130788),
 ('gender_Male', 0.08118152),
 ('race_Caucasian', 0.079858735),
 ('discharge_disposition_id_home', 0.07414091),
 ('diag_1_other', 0.0601671),
 ('diag_1_neoplasms', 0.04985155),
 ('diag_1_injury', 0.045942217),
 ('time_in_hospital', 0.045554116),
 ('age', 0.044185933),
 ('diag_1_respiratory', 0.043300703),
 ('diag_1_digestive', 0.041680664),
 ('number_inpatient', 0.035142235),
 ('admission_type_id_urgent', 0.034784477),
 ('num_procedures', 0.034184977),
 ('discharge_disposition_id_na', 0.025912777),
 ('number_outpatient', 0.023176754),
 ('number_diagnoses', 0.022644263),
 ('diag_1_genitourinary', 0.022518644),
 ('insulin_No', 0.02177657),
 ('number_of_medicine', 0.0175206),
 ('diag_1_diabetes', 0.015366111),
 ('discharge_disposition_id_hospital', 0.014435556),
 ('admission_source_id_urgent', 0.014428959),
 ('admission_source_id_transfer', 0.01409315),
 ('glimepiride_No', 0.013062129),
 ('diag_1_musculoskeletal', 0.011975466),
 ('number_emergency', 0.0112

## parameter tuning

In [61]:
xgbc1 = xgb.XGBClassifier()
xgbc1.set_params(learning_rate=0.1,subsample=0.8,n_estimators=3000,max_depth=6,
                colsample_bytree=1,objective='binary:logistic')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [62]:
#fit X_train, y_train
xgbc1.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [63]:
xgbc1.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)>

In [64]:
thresh=0.5
y_train_pred_proba = xgbc1.predict_proba(X_train)[:,1]
y_valid_pred_proba = xgbc1.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc1.predict_proba(X_test)[:,1]

y_train_pred = xgbc1.predict(X_train)
y_valid_pred = xgbc1.predict(X_valid)
y_test_pred = xgbc1.predict(X_test)

print('Tuned XGBoost')
print('Training:')
print_report(y_train, y_train_pred, y_train_pred_proba)
print('Validation:')
print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)


Tuned XGBoost
Training:
AUC:1.000
False positive rate is 2.058502645175899e-05
 
Validation:
AUC:0.948
False positive rate is 0.023606917375789186
 
Test:
AUC:0.591
False positive rate is 0.8176470588235294
 


In [76]:
confusion_matrix(y_train, y_train_pred)


array([[49126,     1],
       [  867, 48578]])

In [75]:
confusion_matrix(y_valid, y_valid_pred)


array([[12223,   258],
       [ 1492, 10671]])

In [74]:
confusion_matrix(y_test, y_test_pred)


array([[23002,   417],
       [ 2390,    93]])

In [65]:
sorted(zip(X_train.columns,xgbc1.feature_importances_), key= lambda t:t[1], reverse=True)


[('number_inpatient', 0.06680916),
 ('discharge_disposition_id_home', 0.065195575),
 ('race_Caucasian', 0.043022666),
 ('A1Cresult_None', 0.0383296),
 ('gender_Male', 0.037436746),
 ('diag_1_respiratory', 0.030693764),
 ('age', 0.030233378),
 ('diag_1_neoplasms', 0.029993841),
 ('discharge_disposition_id_hospital', 0.027733536),
 ('diag_1_genitourinary', 0.023336885),
 ('discharge_disposition_id_na', 0.023110002),
 ('time_in_hospital', 0.022689397),
 ('diag_1_digestive', 0.022406999),
 ('diag_1_injury', 0.021529803),
 ('diag_1_other', 0.019667866),
 ('num_procedures', 0.019535458),
 ('diag_1_musculoskeletal', 0.018768461),
 ('number_diagnoses', 0.016985325),
 ('diag_1_diabetes', 0.01675759),
 ('admission_type_id_urgent', 0.016176946),
 ('glimepiride_No', 0.014780113),
 ('admission_source_id_urgent', 0.013858564),
 ('number_outpatient', 0.012193782),
 ('admission_source_id_transfer', 0.012043459),
 ('insulin_Up', 0.011254717),
 ('insulin_No', 0.010970293),
 ('admission_source_id_referra

In [66]:
import eli5
eli5.show_weights(xgbc1)


Weight,Feature
0.0668,number_inpatient
0.0652,discharge_disposition_id_home
0.0430,race_Caucasian
0.0383,A1Cresult_None
0.0374,gender_Male
0.0307,diag_1_respiratory
0.0302,age
0.0300,diag_1_neoplasms
0.0277,discharge_disposition_id_hospital
0.0233,diag_1_genitourinary


In [67]:
X_test['predicted_probaility'] = y_test_pred_proba

In [68]:
X_test['predicted_probaility'] = y_test_pred_proba
X_test['actual_readmitted'] = y_test
X_test['predicted_readmitted'] = y_test_pred

In [69]:
X_test.shape

(25902, 94)

In [70]:
X_test.sort_values(by='predicted_probaility')

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,med_dosage_change,...,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes,predicted_probaility,actual_readmitted,predicted_readmitted
21898,-0.046651,-1.138422,-2.137087,0.390898,0.496655,-0.291878,-0.211600,-0.500531,-0.207015,0,...,0,0,0,0,0,1,1,0.000002,0,0
725,-0.046651,-0.801560,0.361645,-0.783594,-0.738640,-0.291878,-0.211600,-0.500531,-1.237394,0,...,0,0,0,0,0,0,1,0.000002,0,0
6463,-1.928080,-0.801560,-2.137087,0.978144,-1.479817,0.498474,-0.211600,-0.500531,-1.752584,0,...,0,0,0,0,0,1,1,0.000002,0,0
22538,0.580493,-1.138422,-2.137087,0.978144,-1.109228,-0.291878,-0.211600,-0.500531,-1.237394,0,...,0,0,0,0,0,0,1,0.000004,0,0
23692,1.207636,-1.138422,-1.015207,0.978144,2.967245,-0.291878,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,0,0,1,0.000005,0,0
5022,-1.300937,-1.138422,-2.137087,0.978144,0.002537,-0.291878,-0.211600,-0.500531,-1.752584,0,...,0,0,0,0,0,1,1,0.000005,0,0
21074,-0.673794,-1.138422,-0.454268,-0.783594,-1.726876,-0.291878,-0.211600,-0.500531,-2.782964,0,...,0,0,0,0,0,0,1,0.000005,0,0
14225,-2.555223,0.209026,-0.556257,-0.196348,-0.368051,-0.291878,-0.211600,-0.500531,-1.752584,1,...,0,0,0,0,0,0,1,0.000006,0,0
880,-3.182366,-0.464698,-0.556257,-0.783594,-1.232758,-0.291878,-0.211600,-0.500531,-3.298153,1,...,0,0,0,0,0,0,1,0.000008,0,0
24389,1.834779,-1.138422,-1.576147,-0.196348,-1.356287,-0.291878,-0.211600,-0.500531,-0.722205,0,...,0,0,0,0,0,0,1,0.000008,0,0


In [71]:
#adding the quantiles as another columns
X_test['Quantile_rank'] = pd.qcut(X_test['predicted_probaility'],10,labels=False)


In [72]:
X_test

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,med_dosage_change,...,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes,predicted_probaility,actual_readmitted,predicted_readmitted,Quantile_rank
0,0.580493,-0.801560,-0.556257,-0.196348,1.855480,-0.291878,-0.211600,0.292829,-1.237394,0,...,0,0,0,0,0,1,0.035946,0,0,5
1,1.207636,2.567059,-1.321175,-0.783594,0.496655,0.498474,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,0,1,0.001300,0,0,0
2,0.580493,-1.138422,-0.658246,2.739882,0.496655,-0.291878,-0.211600,0.292829,0.823365,0,...,0,0,0,0,1,0,0.064248,0,0,6
3,-3.809509,-0.801560,-0.913218,-0.783594,-1.726876,-0.291878,-0.211600,-0.500531,-2.782964,0,...,0,0,0,0,1,0,0.000947,0,0,0
4,-0.046651,0.882750,-0.658246,-0.783594,-0.244522,-0.291878,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,0,1,0.121610,0,0,8
5,0.580493,-0.127836,0.514628,0.390898,-0.491581,-0.291878,-0.211600,0.292829,0.823365,0,...,0,0,0,0,1,1,0.037583,0,0,5
6,0.580493,-0.127836,-0.046311,0.978144,1.237832,-0.291878,-0.211600,-0.500531,-0.722205,0,...,0,0,0,0,1,1,0.559258,0,1,9
7,-0.046651,-0.464698,0.463634,2.152636,0.249596,-0.291878,-0.211600,-0.500531,0.308175,0,...,0,0,0,0,1,1,0.023559,0,0,4
8,0.580493,-0.801560,-1.576147,-0.783594,-0.738640,-0.291878,-0.211600,-0.500531,0.823365,0,...,0,0,0,0,1,1,0.122408,0,0,8
9,0.580493,1.219612,1.789492,-0.196348,0.126067,-0.291878,-0.211600,0.292829,0.823365,1,...,0,0,0,0,0,1,0.160164,0,0,8


In [73]:
X_test.groupby('Quantile_rank')['predicted_probaility'].agg(['min','max','mean'])

,min,max,mean
Quantile_rank,,,
0,0.000002,0.002123,0.000908
1,0.002125,0.005704,0.003784
2,0.005705,0.011097,0.008256
3,0.011098,0.018206,0.014437
4,0.018207,0.028951,0.023254
5,0.028953,0.043758,0.035792
6,0.043762,0.067222,0.054677
7,0.067226,0.107681,0.085507
8,0.107704,0.198705,0.145337
